In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import utilys as ut
import os
from scipy.misc import imread,imresize

In [2]:
width = 20

# gather data paths
subfolds = ut.extend_children('data','')
datafolds = [subfolds[0],subfolds[3]]
alphabets = ut.extend_generation(datafolds,'')
charpaths = ut.extend_generation(alphabets,'')
chars_dataset = [v.split('/')[2]+'/'+v.split('/')[3] for v in charpaths]

# index-value conversion dictionaries for character set
i2v = {i:v for i, v in enumerate(chars_dataset)}
v2i = {v:i for i, v in enumerate(chars_dataset)}

# get size of dataset
mc_dataset = len(charpaths)
print(mc_dataset,'total character classes')

# train split
mc_train = 1423
chars_train = chars_dataset[:mc_train]
classes_train = [v2i[v] for v in chars_train]

# validation split
mc_val = mc_dataset-mc_train
chars_val = chars_dataset[-mc_val:]
classes_val = [v2i[v] for v in chars_val]

print('%s characters assigned for training, %s characters assigned for validation'%(mc_train,mc_val))

1623 total character classes
1423 characters assigned for training, 200 characters assigned for validation


In [3]:
# load images from character paths
imgs_dataset = []
for i, charfold in enumerate(charpaths):
    if i%200==0:
        print('%s/%s character folders loaded'%(i,mc_dataset))
    imgs_dataset.append([ ut.load_image(imgpath,(width,width))/127.5-1 for imgpath in ut.extend_children(charfold,'.png') ] )
# access imgs_dataset [ character index ] [ sample index ] [ row,col ]

# split images between train and validation sets
imgs_train = imgs_dataset[:mc_train]
imgs_val = imgs_dataset[-mc_val:]

0/1623 character folders loaded
200/1623 character folders loaded
400/1623 character folders loaded
600/1623 character folders loaded
800/1623 character folders loaded
1000/1623 character folders loaded
1200/1623 character folders loaded
1400/1623 character folders loaded
1600/1623 character folders loaded


In [4]:
#classes_val, imgs_val = ut.shuffle_xy(classes_val,imgs_val,7)

n_classes = 5
memory_size = 128 # number of features per entry
memory_dim = 40 # number of entries in memory
learning_rate = 1e-3
batch_size = 16

n_inputs = width*width+n_classes
n_hnodes = 200
n_outputs = n_classes
mem_size = memory_size
mem_dim = memory_dim


weights = lambda shape,name: tf.Variable(tf.truncated_normal(shape, stddev=0.1),name=name)
biases = lambda shape,name: tf.Variable(tf.constant(0.1, shape=shape),name=name)
conv2d = lambda x, W: tf.nn.conv2d(x, W, strides=[1, 2, 2, 1], padding='SAME')
max_pool = lambda x: tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [5]:
import tensorflow as tf
from MANN import MANN

X = tf.placeholder(tf.float32, shape=[batch_size, None, width, width])
y = tf.placeholder(tf.float32, shape=[batch_size, None, n_classes])

X_mat = tf.transpose(tf.reshape(X,[batch_size,-1,width*width]),perm=[1,0,2])
y_tr = tf.transpose(y,perm=[1,0,2])
y_tm1 = tf.concat([tf.zeros([1,batch_size,n_classes]),y_tr[:-1,:]],0)

if False:
    W_c1 = weights([5,5,1,2],'Conv1_Weights')
    b_c1 = biases([2],'Conv1_Biases')
    W_c2 = weights([5,5,2,4],'Conv2_Weights')
    b_c2 = biases([4],'Conv2_Biases')

    # 100x100x1
    h_c1 = tf.nn.relu(conv2d(x_image,W_c1)+b_c1)
    # 50x50x2
    h_p1 = max_pool(h_c1)
    # 25x25x2
    h_c2 = tf.nn.relu(conv2d(h_p1,W_c2)+b_c2)
    # 13x13x4
    h_p2 = max_pool(h_c2)
    # 7x7x4
    h_conv = tf.reshape(h_p2,[-1,7*7*4])
    # 196

X_mann = tf.concat([X_mat,y_tm1],-1)

MANN1 = MANN(n_inputs,n_hnodes,n_classes,memory_size,memory_dim,X_mann,batch_size)

In [6]:
n_xh = n_inputs+n_hnodes
n_hr = n_hnodes+mem_dim
W_init = tf.contrib.layers.xavier_initializer()
b_init = tf.constant_initializer(0)
#with tf.variable_scope('Weights', reuse=True):
if False:
    # LSTM Gate Weights
    W_gf = tf.get_variable('W_forget_gate', shape=[n_xh,n_hnodes], initializer=W_init)
    b_gf = tf.get_variable('b_forget_gate', shape=[n_hnodes], initializer=b_init)
    W_gi = tf.get_variable('W_input_gate', shape=[n_xh,n_hnodes], initializer=W_init)
    b_gi = tf.get_variable('b_input_gate', shape=[n_hnodes], initializer=b_init)
    W_go = tf.get_variable('W_output_gate', shape=[n_xh,n_hnodes], initializer=W_init)
    b_go = tf.get_variable('b_output_gate', shape=[n_hnodes], initializer=b_init)
    # LSTM Tanh Weights
    W_u = tf.get_variable('W_remember', shape=[n_xh,n_hnodes], initializer=W_init)
    b_u = tf.get_variable('b_remember', shape=[n_hnodes], initializer=b_init)
    # Controller Weights
    W_kr = tf.get_variable('W_read_key', shape=[n_hnodes,mem_dim], initializer=W_init)
    b_kr = tf.get_variable('b_read_key', shape=[mem_dim], initializer=b_init)
    W_kw = tf.get_variable('W_write_key', shape=[n_hnodes,mem_dim], initializer=W_init)
    b_kw = tf.get_variable('b_write_key', shape=[mem_dim], initializer=b_init)
    alpha = tf.get_variable('sigmoid_alpha', shape=[1,1], initializer=b_init)
    # Logit Weights
    W_o = tf.get_variable('W_output', shape=[n_hr,n_outputs], initializer=W_init)
    b_o = tf.get_variable('b_output', shape=[n_outputs], initializer=b_init)
    pass
#thetas = [W_gf,b_gf,W_gi,b_gi,W_go,b_go,W_u,b_u,W_kr,b_kr,W_kw,alpha,b_kw,W_o,b_o]

In [7]:
M, h, c, wu, wr, r, o = MANN1

xentropy = tf.nn.softmax_cross_entropy_with_logits(labels=y_tr,logits=o)
loss = tf.reduce_mean(xentropy)

trainable_variables = tf.trainable_variables()
optimizer = tf.train.AdamOptimizer(learning_rate)
grads, _ = tf.clip_by_global_norm(tf.gradients(loss,trainable_variables),30)
train_step = optimizer.apply_gradients(zip(grads,trainable_variables))

correct_prediction = tf.equal(tf.argmax(o, -1), tf.argmax(y_tr, -1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

(?, 16, 405) (?, 16, 5) <unknown>


In [8]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()
savedir = './saved_models_omniglot/Mod1'

In [10]:
import time
import os

n_epochs = 400

if True:
    saver.restore(sess, os.path.join(savedir,'400/Model.ckpt'))

with sess.as_default():
    for epoch in range(n_epochs):
        print('Epoch {} starting..'.format(epoch+1))
        epoch_start = time.time()
        classes_epoch, imgs_epoch = ut.shuffle_xy(classes_train,imgs_train)
        
        n_batches = len(classes_epoch)//(n_classes*batch_size)
        for batch in range(n_batches):
            classes_batch = classes_epoch[batch*n_classes*batch_size:(batch+1)*n_classes*batch_size]
            imgs_batch = imgs_epoch[batch*n_classes*batch_size:(batch+1)*n_classes*batch_size]
            
            Xl_batch, yl_batch = [], []
            for episode in range(batch_size):
                imgs_ep = imgs_batch[episode*n_classes:(episode+1)*n_classes]
                Xl_ep, yl_ep = [], []
                for ind, cat in enumerate(imgs_ep):
                    for arr in cat:
                        Xl_ep.append(arr)
                        yl_ep.append(ut.one_hot(ind,n_classes))
                Xl_shuff, yl_shuff = ut.shuffle_xy(Xl_ep,yl_ep)
                X_arr, y_arr = np.asarray(Xl_shuff), np.asarray(yl_shuff)
                Xl_batch.append(X_arr)
                yl_batch.append(y_arr)
            X_train, y_train = np.asarray(Xl_batch), np.asarray(yl_batch)
            
            _, loss_train, accuracy_train = sess.run((train_step,loss,accuracy),feed_dict={X:X_train,y:y_train})
            print('Epoch %d'%(epoch+1),'/ Batch (%d/%d)'%(batch+1,n_batches),'/ Accuracy %.4f / Loss %f'%(accuracy_train,loss_train))
            
        epoch_end = time.time()
        time_elapsed = epoch_end-epoch_start
        print('Epoch {} complete,'.format(epoch+1),time_elapsed,'seconds elapsed')
        if (epoch+1)%40 == 0:
            iterdir = os.path.join(savedir,str(epoch+1))
            if not os.path.exists(iterdir):
                os.makedirs(iterdir)
            saver.save(sess,'{}/Model.ckpt'.format(iterdir))
        print()

INFO:tensorflow:Restoring parameters from ./saved_models_omniglot/Mod1\400/Model.ckpt
Epoch 1 starting..
Epoch 1 / Batch (1/17) / Accuracy 0.7113 / Loss 0.671945
Epoch 1 / Batch (2/17) / Accuracy 0.7450 / Loss 0.644811
Epoch 1 / Batch (3/17) / Accuracy 0.7237 / Loss 0.671712
Epoch 1 / Batch (4/17) / Accuracy 0.7106 / Loss 0.681302
Epoch 1 / Batch (5/17) / Accuracy 0.7369 / Loss 0.638603
Epoch 1 / Batch (6/17) / Accuracy 0.7556 / Loss 0.609996
Epoch 1 / Batch (7/17) / Accuracy 0.7437 / Loss 0.645145
Epoch 1 / Batch (8/17) / Accuracy 0.7294 / Loss 0.658106
Epoch 1 / Batch (9/17) / Accuracy 0.7256 / Loss 0.660708
Epoch 1 / Batch (10/17) / Accuracy 0.7344 / Loss 0.639564
Epoch 1 / Batch (11/17) / Accuracy 0.7244 / Loss 0.659034
Epoch 1 / Batch (12/17) / Accuracy 0.6950 / Loss 0.724729
Epoch 1 / Batch (13/17) / Accuracy 0.7506 / Loss 0.624792
Epoch 1 / Batch (14/17) / Accuracy 0.7344 / Loss 0.651112
Epoch 1 / Batch (15/17) / Accuracy 0.7212 / Loss 0.650918
Epoch 1 / Batch (16/17) / Accuracy

Epoch 8 / Batch (14/17) / Accuracy 0.7669 / Loss 0.606161
Epoch 8 / Batch (15/17) / Accuracy 0.7194 / Loss 0.693386
Epoch 8 / Batch (16/17) / Accuracy 0.7381 / Loss 0.612737
Epoch 8 / Batch (17/17) / Accuracy 0.7800 / Loss 0.574044
Epoch 8 complete, 6.74535608291626 seconds elapsed

Epoch 9 starting..
Epoch 9 / Batch (1/17) / Accuracy 0.7531 / Loss 0.581178
Epoch 9 / Batch (2/17) / Accuracy 0.7206 / Loss 0.648977
Epoch 9 / Batch (3/17) / Accuracy 0.7844 / Loss 0.546066
Epoch 9 / Batch (4/17) / Accuracy 0.7713 / Loss 0.606559
Epoch 9 / Batch (5/17) / Accuracy 0.7656 / Loss 0.560695
Epoch 9 / Batch (6/17) / Accuracy 0.7681 / Loss 0.547315
Epoch 9 / Batch (7/17) / Accuracy 0.7619 / Loss 0.593824
Epoch 9 / Batch (8/17) / Accuracy 0.7237 / Loss 0.613027
Epoch 9 / Batch (9/17) / Accuracy 0.7431 / Loss 0.616972
Epoch 9 / Batch (10/17) / Accuracy 0.7856 / Loss 0.563586
Epoch 9 / Batch (11/17) / Accuracy 0.7569 / Loss 0.609299
Epoch 9 / Batch (12/17) / Accuracy 0.7206 / Loss 0.658705
Epoch 9 / 

Epoch 16 / Batch (9/17) / Accuracy 0.7937 / Loss 0.548923
Epoch 16 / Batch (10/17) / Accuracy 0.7294 / Loss 0.633567
Epoch 16 / Batch (11/17) / Accuracy 0.7887 / Loss 0.544576
Epoch 16 / Batch (12/17) / Accuracy 0.7163 / Loss 0.674455
Epoch 16 / Batch (13/17) / Accuracy 0.7550 / Loss 0.584072
Epoch 16 / Batch (14/17) / Accuracy 0.7369 / Loss 0.641047
Epoch 16 / Batch (15/17) / Accuracy 0.7719 / Loss 0.597684
Epoch 16 / Batch (16/17) / Accuracy 0.7569 / Loss 0.612319
Epoch 16 / Batch (17/17) / Accuracy 0.7450 / Loss 0.603866
Epoch 16 complete, 6.946115970611572 seconds elapsed

Epoch 17 starting..
Epoch 17 / Batch (1/17) / Accuracy 0.7375 / Loss 0.643184
Epoch 17 / Batch (2/17) / Accuracy 0.7306 / Loss 0.656599
Epoch 17 / Batch (3/17) / Accuracy 0.7725 / Loss 0.595611
Epoch 17 / Batch (4/17) / Accuracy 0.7538 / Loss 0.610237
Epoch 17 / Batch (5/17) / Accuracy 0.7894 / Loss 0.546325
Epoch 17 / Batch (6/17) / Accuracy 0.7656 / Loss 0.571148
Epoch 17 / Batch (7/17) / Accuracy 0.7887 / Loss

Epoch 24 / Batch (3/17) / Accuracy 0.8250 / Loss 0.466747
Epoch 24 / Batch (4/17) / Accuracy 0.7681 / Loss 0.563182
Epoch 24 / Batch (5/17) / Accuracy 0.7481 / Loss 0.607339
Epoch 24 / Batch (6/17) / Accuracy 0.7544 / Loss 0.598065
Epoch 24 / Batch (7/17) / Accuracy 0.7962 / Loss 0.522475
Epoch 24 / Batch (8/17) / Accuracy 0.8119 / Loss 0.492900
Epoch 24 / Batch (9/17) / Accuracy 0.7769 / Loss 0.558326
Epoch 24 / Batch (10/17) / Accuracy 0.7600 / Loss 0.573125
Epoch 24 / Batch (11/17) / Accuracy 0.8131 / Loss 0.498918
Epoch 24 / Batch (12/17) / Accuracy 0.7656 / Loss 0.591792
Epoch 24 / Batch (13/17) / Accuracy 0.7219 / Loss 0.650938
Epoch 24 / Batch (14/17) / Accuracy 0.7656 / Loss 0.586672
Epoch 24 / Batch (15/17) / Accuracy 0.7906 / Loss 0.553954
Epoch 24 / Batch (16/17) / Accuracy 0.7613 / Loss 0.628721
Epoch 24 / Batch (17/17) / Accuracy 0.7475 / Loss 0.606786
Epoch 24 complete, 6.047562122344971 seconds elapsed

Epoch 25 starting..
Epoch 25 / Batch (1/17) / Accuracy 0.7419 / Loss

Epoch 31 / Batch (16/17) / Accuracy 0.7487 / Loss 0.606316
Epoch 31 / Batch (17/17) / Accuracy 0.8025 / Loss 0.519609
Epoch 31 complete, 6.210492372512817 seconds elapsed

Epoch 32 starting..
Epoch 32 / Batch (1/17) / Accuracy 0.7944 / Loss 0.552802
Epoch 32 / Batch (2/17) / Accuracy 0.7962 / Loss 0.534948
Epoch 32 / Batch (3/17) / Accuracy 0.7750 / Loss 0.557986
Epoch 32 / Batch (4/17) / Accuracy 0.7437 / Loss 0.622557
Epoch 32 / Batch (5/17) / Accuracy 0.7925 / Loss 0.515151
Epoch 32 / Batch (6/17) / Accuracy 0.7206 / Loss 0.640117
Epoch 32 / Batch (7/17) / Accuracy 0.7719 / Loss 0.607279
Epoch 32 / Batch (8/17) / Accuracy 0.7525 / Loss 0.615771
Epoch 32 / Batch (9/17) / Accuracy 0.8106 / Loss 0.504405
Epoch 32 / Batch (10/17) / Accuracy 0.7663 / Loss 0.600492
Epoch 32 / Batch (11/17) / Accuracy 0.7525 / Loss 0.631222
Epoch 32 / Batch (12/17) / Accuracy 0.7819 / Loss 0.543282
Epoch 32 / Batch (13/17) / Accuracy 0.7819 / Loss 0.555128
Epoch 32 / Batch (14/17) / Accuracy 0.7931 / Loss 

...

Epoch 365 / Batch (13/17) / Accuracy 0.8425 / Loss 0.404223
Epoch 365 / Batch (14/17) / Accuracy 0.8144 / Loss 0.458892
Epoch 365 / Batch (15/17) / Accuracy 0.8131 / Loss 0.451438
Epoch 365 / Batch (16/17) / Accuracy 0.8500 / Loss 0.378467
Epoch 365 / Batch (17/17) / Accuracy 0.8331 / Loss 0.450083
Epoch 365 complete, 5.170769453048706 seconds elapsed

Epoch 366 starting..
Epoch 366 / Batch (1/17) / Accuracy 0.8425 / Loss 0.396252
Epoch 366 / Batch (2/17) / Accuracy 0.8087 / Loss 0.491229
Epoch 366 / Batch (3/17) / Accuracy 0.8200 / Loss 0.446925
Epoch 366 / Batch (4/17) / Accuracy 0.8112 / Loss 0.479899
Epoch 366 / Batch (5/17) / Accuracy 0.8519 / Loss 0.371181
Epoch 366 / Batch (6/17) / Accuracy 0.8275 / Loss 0.412489
Epoch 366 / Batch (7/17) / Accuracy 0.8375 / Loss 0.393839
Epoch 366 / Batch (8/17) / Accuracy 0.8175 / Loss 0.449383
Epoch 366 / Batch (9/17) / Accuracy 0.8356 / Loss 0.419550
Epoch 366 / Batch (10/17) / Accuracy 0.8125 / Loss 0.456242
Epoch 366 / Batch (11/17) / Accur

Epoch 373 / Batch (5/17) / Accuracy 0.8175 / Loss 0.491845
Epoch 373 / Batch (6/17) / Accuracy 0.8450 / Loss 0.415164
Epoch 373 / Batch (7/17) / Accuracy 0.8450 / Loss 0.407657
Epoch 373 / Batch (8/17) / Accuracy 0.8725 / Loss 0.335671
Epoch 373 / Batch (9/17) / Accuracy 0.8569 / Loss 0.384409
Epoch 373 / Batch (10/17) / Accuracy 0.8456 / Loss 0.407235
Epoch 373 / Batch (11/17) / Accuracy 0.8119 / Loss 0.464250
Epoch 373 / Batch (12/17) / Accuracy 0.8213 / Loss 0.464025
Epoch 373 / Batch (13/17) / Accuracy 0.8081 / Loss 0.502483
Epoch 373 / Batch (14/17) / Accuracy 0.8556 / Loss 0.363706
Epoch 373 / Batch (15/17) / Accuracy 0.8813 / Loss 0.339943
Epoch 373 / Batch (16/17) / Accuracy 0.8188 / Loss 0.452389
Epoch 373 / Batch (17/17) / Accuracy 0.8350 / Loss 0.430583
Epoch 373 complete, 5.484385967254639 seconds elapsed

Epoch 374 starting..
Epoch 374 / Batch (1/17) / Accuracy 0.8313 / Loss 0.421024
Epoch 374 / Batch (2/17) / Accuracy 0.8081 / Loss 0.475794
Epoch 374 / Batch (3/17) / Accu

Epoch 380 / Batch (15/17) / Accuracy 0.8562 / Loss 0.376095
Epoch 380 / Batch (16/17) / Accuracy 0.8106 / Loss 0.487519
Epoch 380 / Batch (17/17) / Accuracy 0.8163 / Loss 0.461185
Epoch 380 complete, 5.770671606063843 seconds elapsed

Epoch 381 starting..
Epoch 381 / Batch (1/17) / Accuracy 0.8338 / Loss 0.432692
Epoch 381 / Batch (2/17) / Accuracy 0.8338 / Loss 0.406721
Epoch 381 / Batch (3/17) / Accuracy 0.8450 / Loss 0.406309
Epoch 381 / Batch (4/17) / Accuracy 0.8269 / Loss 0.440055
Epoch 381 / Batch (5/17) / Accuracy 0.8150 / Loss 0.454356
Epoch 381 / Batch (6/17) / Accuracy 0.8537 / Loss 0.368425
Epoch 381 / Batch (7/17) / Accuracy 0.8381 / Loss 0.404754
Epoch 381 / Batch (8/17) / Accuracy 0.8406 / Loss 0.404923
Epoch 381 / Batch (9/17) / Accuracy 0.8494 / Loss 0.359417
Epoch 381 / Batch (10/17) / Accuracy 0.8487 / Loss 0.424374
Epoch 381 / Batch (11/17) / Accuracy 0.8400 / Loss 0.401527
Epoch 381 / Batch (12/17) / Accuracy 0.8238 / Loss 0.440315
Epoch 381 / Batch (13/17) / Accur

Epoch 388 / Batch (7/17) / Accuracy 0.8375 / Loss 0.402202
Epoch 388 / Batch (8/17) / Accuracy 0.8194 / Loss 0.434908
Epoch 388 / Batch (9/17) / Accuracy 0.8575 / Loss 0.384884
Epoch 388 / Batch (10/17) / Accuracy 0.8100 / Loss 0.479487
Epoch 388 / Batch (11/17) / Accuracy 0.8531 / Loss 0.396593
Epoch 388 / Batch (12/17) / Accuracy 0.8419 / Loss 0.413618
Epoch 388 / Batch (13/17) / Accuracy 0.8400 / Loss 0.415553
Epoch 388 / Batch (14/17) / Accuracy 0.8219 / Loss 0.432781
Epoch 388 / Batch (15/17) / Accuracy 0.8462 / Loss 0.403051
Epoch 388 / Batch (16/17) / Accuracy 0.8288 / Loss 0.431000
Epoch 388 / Batch (17/17) / Accuracy 0.8319 / Loss 0.424539
Epoch 388 complete, 5.829784870147705 seconds elapsed

Epoch 389 starting..
Epoch 389 / Batch (1/17) / Accuracy 0.8288 / Loss 0.411104
Epoch 389 / Batch (2/17) / Accuracy 0.8081 / Loss 0.456704
Epoch 389 / Batch (3/17) / Accuracy 0.8338 / Loss 0.412193
Epoch 389 / Batch (4/17) / Accuracy 0.8225 / Loss 0.437441
Epoch 389 / Batch (5/17) / Accu

Epoch 395 / Batch (17/17) / Accuracy 0.8381 / Loss 0.426970
Epoch 395 complete, 6.016657829284668 seconds elapsed

Epoch 396 starting..
Epoch 396 / Batch (1/17) / Accuracy 0.8462 / Loss 0.394463
Epoch 396 / Batch (2/17) / Accuracy 0.8625 / Loss 0.362781
Epoch 396 / Batch (3/17) / Accuracy 0.8288 / Loss 0.430087
Epoch 396 / Batch (4/17) / Accuracy 0.8369 / Loss 0.404840
Epoch 396 / Batch (5/17) / Accuracy 0.8169 / Loss 0.480826
Epoch 396 / Batch (6/17) / Accuracy 0.8512 / Loss 0.385863
Epoch 396 / Batch (7/17) / Accuracy 0.8712 / Loss 0.330523
Epoch 396 / Batch (8/17) / Accuracy 0.8506 / Loss 0.376084
Epoch 396 / Batch (9/17) / Accuracy 0.8294 / Loss 0.439016
Epoch 396 / Batch (10/17) / Accuracy 0.8188 / Loss 0.446438
Epoch 396 / Batch (11/17) / Accuracy 0.8431 / Loss 0.409443
Epoch 396 / Batch (12/17) / Accuracy 0.8819 / Loss 0.343340
Epoch 396 / Batch (13/17) / Accuracy 0.8481 / Loss 0.376595
Epoch 396 / Batch (14/17) / Accuracy 0.8487 / Loss 0.404466
Epoch 396 / Batch (15/17) / Accur